In [1]:
# !pip install cv2
!pip install dlib
import os
import pickle
import numpy as np
import cv2
import face_recognition
import cvzone
import firebase_admin
from firebase_admin import credentials
from firebase_admin import db
from firebase_admin import storage
import numpy as np
from datetime import datetime

In [ ]:
#ACTIVACIÓN DE LA CÁMARA PARA LAS CAPTURAS Y SU RESOLUCIÓN, CON 0 LA ABRIMOS, CON 1 LA CERRAMOS
#----------------------------------------------------------
#----------------------------------------------------------
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

#DECLARAMOS NUESTRA IMAGEN DE FONDO
#----------------------------------------------------------
imgBackground = cv2.imread('Resources/background.png')


#IMPORTANDO EL MODO DE IMÁGENES EN UNA LISTA
#----------------------------------------------------------
folderModePath = 'Resources/Modes'
modePathList = os.listdir(folderModePath)
imgModeList = []
for path in modePathList:
    imgModeList.append(cv2.imread(os.path.join(folderModePath,path)))

#print(len(imgModelList))

#LOAD THE ENCODING FILE
print("Loading Encode File ...")
file = open("EncodeFile.p","rb")
encodeListKnownWithIds = pickle.load(file)
file.close()
encodeListKnown, studentIds = encodeListKnownWithIds
# print(studentIds)
print("Encode File Loaded")

    
while True:
    sucess, img = cap.read()

#REDUCTION IMAGES SIZE TO QUICKLY PROCESS
    imgS = cv2.resize(img, (0, 0), None, 0.25, 0.25)
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)

    faceCurFrame = face_recognition.face_locations(imgS)
    encodeCurFrame = face_recognition.face_encodings(imgS, faceCurFrame)

#SUPERPONEMOS EL FONDO CON SUS MEDIDAS 
#----------------------------------------------------------
    imgBackground[162:162 + 480, 55:55 + 640] = img 
#----------------------------------------------------------
#Y AÑADIMOS TAMBIÉN LAS OTRAS IMÁGENES QUE INSERTAREMOS PERO CON OTRAS MEDIDAS  
    imgBackground[44:44 + 633, 808:808 + 414] = imgModeList[0]
    
#COMPARE IMAGES ONE TO ONE. 
    for encodeFace, faceLoc in zip (encodeCurFrame, faceCurFrame):
        matches = face_recognition.compare_faces(encodeListKnown, encodeFace)
        faceDis = face_recognition.face_distance(encodeListKnown, encodeFace)
        print("matches", matches)
        print("faceDis", faceDis)
        
        matchIndex = np.argmin(faceDis)
        # print("Match Index", matchIndex)
        
        if matches[matchIndex]:
            # print("Known Face Detected")
            # print(studentIds[matchIndex])
            
#CREAMOS EL CUADRADO DELIMITADOR PARA CAPTURAR
            y1, x2, y2, x1 = faceLoc
            y1, x2, y2, x1 = y1 * 4, x2 *4, y2 *4, x1* 4
            bbox = 55 + x1, 162 + y1, x2 - x1, y2 - y1
            cvzone.cornerRect(imgBackground,bbox,rt=0)
        
#SALE LA CÁMARA Y NUESTRO PHOTOCALL DE FONDO
#----------------------------------------------------------
    # cv2.imshow("Webcam", img)
    cv2.imshow("Face Attendance", imgBackground)
    cv2.waitKey(1)